本文档是一个介绍深圳市肺炎的小区地图，其中小区数据来源于深圳卫健委官方网站，经纬度数据使用了百度地图地理编码api

In [1]:
import pandas as pd 
import numpy as np 

#用来通过http 调用百度接口
from urllib import request,parse
# 解析接口返回的信息
import json

#画图
from pyecharts.charts import  Geo

### 地理编码
地理编码是指通过一个文本地址，找到其对应的经纬度,我们利用百度的地理编码接口完成这个事情
接口使用参考文档：http://lbs.baidu.com/index.php?title=webapi/guide/webservice-geocoding

注：这里也可以用pyecharts中的Geo().get_coordinate( "深圳市新洲花园") 方法

In [3]:
def getLocation(address):
    lat = 0 
    lng = 0 
    url = "http://api.map.baidu.com/geocoding/v3/"
    parms = {
    'address':address,
    'output':'json',
    'ak':'PCiCHRK1snUjMmjOzTBEnBGx2Px8lhQ9',
    'callback':'showLocation'}
    
    queryString=parse.urlencode(parms)
    u = request.urlopen(url +'?'+queryString)

    try:
        resp=u.read().decode('utf-8').replace('showLocation&&showLocation(','').replace(')','')
        location = json.loads(resp).get("result").get("location")
        lat = location.get("lat")
        lng = location.get("lng")
    except Exception:
        pass
    
    return  [lng,lat]
    



location = getLocation('深圳市福田区华富街道福莲花园')
print(location)

[114.07558293841585, 22.551547767201782]


将从深圳卫健委获取的数据加载成dataframe，然后利用上面的函数获取经纬度

In [6]:
df = pd.read_csv("ncov2019.csv")

df["detail_address"] = df.city+df.area+df.address
#df["loaction"] = df.detail_address.map(getLocation)

df.to_csv("ncov2019_location.csv")

df.head()

,city,area,address,publishdate,detail_address,loaction
0,深圳市,南山区,沙河街道沙河东路世纪村,20200131,深圳市南山区沙河街道沙河东路世纪村,"[113.97225956920066, 22.53954100311121]"
1,深圳市,南山区,鸿丰大酒店,20200131,深圳市南山区鸿丰大酒店,"[113.91880922357747, 22.51821630457622]"
2,深圳市,南山区,科技园华润城,20200131,深圳市南山区科技园华润城,"[113.96138108947842, 22.551806693240444]"
3,深圳市,南山区,华侨城香山美墅,20200131,深圳市南山区华侨城香山美墅,"[113.97604035742981, 22.556631072162002]"
4,深圳市,南山区,大冲宿禾服务式公寓,20200131,深圳市南山区大冲宿禾服务式公寓,"[113.95870395939846, 22.547376011284285]"


### 简单地图撒点
本次我们的地图将要使用pyecharts包
pyechart参考文档：https://pyecharts.org/#/zh-cn/geography_charts?id=geo%ef%bc%9a%e5%9c%b0%e7%90%86%e5%9d%90%e6%a0%87%e7%b3%bb


我们先用pyechars 做一个简单的地图撒点，即把深圳市所有确诊患者出现过的地方在深圳地图上描绘出来


In [85]:
# 先把经纬度处理成一个json串

loc_dic = {}
indexs = []
for area,loc in zip(df.address,df.loaction):
    loc_dic[area] = loc
    indexs.append(area)

values = np.ones(121).tolist()

#深圳所有的区
district = df.area.unique().tolist()
dis_values = np.zeros(len(district)).tolist()

#存入一个本地文件
with open("loc_json.json",'w') as f:
    json.dump(loc_dic,f)

In [101]:
from pyecharts.globals import ChartType
from pyecharts import options as opts
from pyecharts.charts import Geo


def geo_effectscatter() -> Geo:
    c = (
        Geo()
        .add_schema(maptype="深圳")
        .add_coordinate_json( 'loc_json.json' )
        
        .add(
            "确诊小区",
            [list(z) for z in zip(indexs, values)],
            type_=ChartType.EFFECT_SCATTER
        )
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
        .set_global_opts(title_opts=opts.TitleOpts(title="深圳新型肺炎分布图"))
    )
    return c

geo_effectscatter().render_notebook()

### 问题
1. 每个区的名字没有显示上来
2. 地图上没有道路等详细信息
3. 没有体现出时间的纬度作用
